# AI used in Crossword Generator

## Background

Crossword Generator can be framed as a constrant satisfaction problem, CSP. A constraint satisfaction problem is defined by the following:

1. X is a set of variables, X1, X2,..., Xn, each defined by a finite domain, D1, D2, ..., Dn of possible values.
2. C is a set of constraints, C1, C2, ... Cn. The constraints limit the set of possible values that can be present in each domain.

The goal of a constraint satisfaction is to search for a solution - an assignment of value(s) to each vvariable that satisfies the constraints.

Consider the following:
![crosswordpic](assets/crosswordpic.png)

Each variable (crossword vaeriable) is an object with the following properties:
- Direction - across or down
- Length - number of squares it occupies
- Starting row & col: represented as [i, j]

### Types of Constraints
Unary constraint: a type of constraint that involves a restriction on one variable. Eg. the length of a word == 3.

Binary constraint: a type of constraint that involves a restriction on two variables. Eg. The word assigned to X1 and the word assigned to X2 must overlap at a certain square.


### Viewing a CSP as a search problem:

- Initial state: State which all the variables are unassigned.
- Actions: Assign a value to a variable, eg assignment[var] = value
- Transition model: The result of state + action.
- Goal test: Whether the problem fulfills the constraints, defined by the function `consistent`.
- Path cost function: Time taken to solve the constraint satisfaction probblem.

Instead of a Breadth First Search (BFS) or Depth First Search (DFS) algorithm, the `backtrack` algorithm is utilized.

Courtesy of Cornell University.

In [ ]:
#imports

import sys
import itertools
import random
import numpy as np

from crossword import *

## Logistics

In [ ]:
class CrosswordCreator():

    def __init__(self, crossword):
        """
        Create new CSP crossword generate.
        """
        self.crossword = crossword
        self.domains = {
            var: self.crossword.words.copy()
            for var in self.crossword.variables
        }

    def letter_grid(self, assignment):
        """
        Return 2D array representing a given assignment.
        """
        letters = [
            [None for _ in range(self.crossword.width)]
            for _ in range(self.crossword.height)
        ]
        for variable, word in assignment.items():
            direction = variable.direction
            for k in range(len(word)):
                i = variable.i + (k if direction == Variable.DOWN else 0)
                j = variable.j + (k if direction == Variable.ACROSS else 0)
                letters[i][j] = word[k]
        return letters

    def print(self, assignment):
        """
        Print crossword assignment to the terminal.
        """
        letters = self.letter_grid(assignment)
        for i in range(self.crossword.height):
            for j in range(self.crossword.width):
                if self.crossword.structure[i][j]:
                    print(letters[i][j] or " ", end="")
                else:
                    print("█", end="")
            print()

    def save(self, assignment, filename):
        """
        Save crossword assignment to an image file.
        """
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        interior_size = cell_size - 2 * cell_border
        letters = self.letter_grid(assignment)

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.crossword.width * cell_size,
             self.crossword.height * cell_size),
            "black"
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 80)
        draw = ImageDraw.Draw(img)

        for i in range(self.crossword.height):
            for j in range(self.crossword.width):

                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                if self.crossword.structure[i][j]:
                    draw.rectangle(rect, fill="white")
                    if letters[i][j]:
                        w, h = draw.textsize(letters[i][j], font=font)
                        draw.text(
                            (rect[0][0] + ((interior_size - w) / 2),
                             rect[0][1] + ((interior_size - h) / 2) - 10),
                            letters[i][j], fill="black", font=font
                        )

        img.save(filename)

    def solve(self):
        """
        Enforce node and arc consistency, and then solve the CSP.
        """
        self.enforce_node_consistency()
        self.ac3()
        return self.backtrack(dict())

# Node Consistency

`enforce_node_consistency` ensures that the unary constraints of all variables is satisfied - in this case, all of the values in a crossword variable's domain must equate the crossword variable's length.

In [ ]:
#add class function

    def enforce_node_consistency(self):
        """
        Enforcing node consistency
        """
        doms = {}
        
        for var, values in self.domains.items():
            doms[var] = set()
            
            for value in values:
                if len(value) == var.length:
                    doms[var].add(value)
                    
        self.domains = doms

# Arc consistency

`revise` and `ac3` enforces arc consistency by ensuring that all the binary constraints of the problems are fulfilled - a word in the domain of variable x must overlap with a word in the domain of variable y at the appropriate location, determined by `self.crossword.overlaps[x,y]`.

In [ ]:
     def revise(self, x, y):
        """
        Make variable `x` arc consistent with variable `y`.
        To do so, remove values from `self.domains[x]` for which there is no
        possible corresponding value for `y` in `self.domains[y]`.

        Return True if a revision was made to the domain of `x`; return
        False if no revision was made.
        """
        
        if not self.crossword.overlaps[x,y]:
            return False
        
    
        i, j = self.crossword.overlaps[x,y]
        
        
        for x_word in self.domains[x]:
            to_remove = True
            
            for y_word in self.domains[y]:
                if x_word[i] == y_word[j]:
                    to_remove = False
                    
            if to_remove:
                self.domains[x].remove(x_word)
                return True
            
        return False

`ac3` uses the function `revise` to enforce arc consistency. It also uses a queue data structure - which is a first-in-first-out (FIFO) data structure. Each combination of arcs is tested for arc consistency. If all of them are arc consistent, the function returns `True`.

In [ ]:
  def ac3(self, arcs=None):
        """
        Update self.domains to ensure that it is arc consistent.
        
        Return True if arc consistent enforced, false if not.
        """
        
        if arcs == None:
            arcs = list(itertools.combinations(self.domains.keys(), 2))
            
        
        while len(arcs) != 0:
            
            (x, y) = arcs.pop(0)
            
            if self.revise(x, y):
                
                if len(self.domains[x]) == 0:
                    return False
                
                for z in self.crossword.neighbors(x) - self.domains[y]:
                    arcs.append((z, x))
        
        return True  


# Backtrack and related algorithms

The main focus of this section would be on the backtrack algorithm, which is a recursive search algorithm adapted for constraint optimization problems. It recursively selects unassigned variables, assigns them to `assignment` and creates a transition state, `assignment_copy`. If the assignment is `consistent`, the algorithm will repeat recursively until all the variables are assigned.

If the assignment is not `consistent`, the transition state will be deleted and the recursion will continue.

If `assignment_complete` is `True`, the backtrack algorithm will return the completed assignment.

Note: the criteria for consistent is defined as follows:
1. All variables are `node consistent`
2. All variables are `arc consistent` with one another
3. All words assigned to each crossword variable are `unique` (meaning that you cannot have repeated words for your crossword puzzle).

In [ ]:
    def assignment_complete(self, assignment):
        """
        Return True if `assignment` is complete (i.e., assigns a value to each
        crossword variable); return False otherwise.
        """
        if len(assignment.keys()) - len(self.domains.keys()) == 0:
            return True
        return False

In [ ]:
    def consistent(self, assignment):
        """
        Return True if `assignment` is consistent (i.e., words fit in crossword
        puzzle without conflicting characters); return False otherwise.
        """
        
        #node consistent
        for var, value in assignment.items():
            if type(value) == str and len(value) != var.length:
                return False
            elif type(value) == set:
                for v in value:
                    if len(v) != var.length:
                        return False
                    
        
        #values must be unique
        unique_words = set()
        
        for var in assignment:
            if assignment[var] not in unique_words:
                unique_words.add(assignment[var])
            else:
                return False
            
        #arc consistent
        for var1, var2 in itertools.combinations(assignment, 2):
            if self.crossword.overlaps[var1, var2]:
                i, j = self.crossword.overlaps[var1, var2]
                if assignment[var1][i] != assignment[var2][j]:
                    return False
        
        return True

In [ ]:
     def backtrack(self, assignment):
        """
        Using Backtracking Search, take as input a partial assignment for the
        crossword and return a complete assignment if possible to do so.

        `assignment` is a mapping from variables (keys) to words (values).

        If no assignment is possible, return None.
        """
        
        
        if self.assignment_complete(assignment):
            return assignment
        
        var = self.select_unassigned_variable(assignment)
        
        for value in self.domains[var]:
            
            assignment_copy = assignment.copy()
            assignment_copy[var] = value
            
            if self.consistent(assignment_copy):
                result = self.backtrack(assignment_copy)
                
                if result != None:
                    return result
                
                del assignment_copy[var]
                
        return None

# Optimization

`order_domain_values` relies on the least constraining values heuristic. The least constraining values heuristic refers to the value in the variable's domain which constraints other variables the least.

`select_unassigned_variables` relies on the minimum remaining value heuristic and then the degree heuristic. The minimum remaining value heuristic chooses the variable with the least number of values in the domain. If there is a tie, the tiebreaker utilized will be the degree heuristic, which chooses the variable with the most number of neighbors (variables that are connected to it).


For the documents "data/structure2.txt" and "data/words2.txt", the path cost function (time taken to solve the problem) was reduced by 2 orders of magnitude:

![optimization](assets/optimization.png)

In [ ]:
    def order_domain_values(self, var, assignment):
        """
        Return a list of values in the domain of `var`, in order by
        the number of values they rule out for neighboring variables.
        The first value in the list, for example, should be the one
        that rules out the fewest values among the neighbors of `var`.
        """
        
        #optimization
        
        if var in assignment.keys():
            print("variable already assigned!")
            pass
        
        count = {}
        
        for value in self.domains[var]:
            count[value] = 0
            for nb in self.crossword.neighbors(var):
                if nb not in assignment.keys():                    
                    if value in self.domains[nb]:
                        count[value] += 1
                    
                    i, j = self.crossword.overlaps[var, nb]
                    for nb_value in self.domains[nb]:
                        if value[i] != nb_value[j]:
                            count[value] += 1
        
        return sorted(count, key = count.get)

In [ ]:
    def select_unassigned_variable(self, assignment):
        """
        Return an unassigned variable not already part of `assignment`.
        Choose the variable with the minimum number of remaining values
        in its domain. If there is a tie, choose the variable with the highest
        degree. If there is a tie, any of the tied variables are acceptable
        return values.
        """
        
        unassigned = list(self.domains.keys() - assignment.keys())
        
        
        #min number of remaining values
        min_value = np.inf
        min_list = []
        
        for var in unassigned:
            if len(self.domains[var]) < min_value:
                min_value = len(self.domains[var])
                
        for var in unassigned:
            if len(self.domains[var]) == min_value:
                min_list.append(var)
                
        if len(min_list) == 1:
            return min_list[0]
        else:
            
            max_nb_count = -np.inf
            max_list = []
            
            for var in min_list:
                if len(self.crossword.neighbors(var)) > max_nb_count:
                    max_nb_count = len(self.crossword.neighbors(var))
                    
            for var in min_list:
                if len(self.crossword.neighbors(var)) == max_nb_count:
                    max_list.append(var)
                    
            if len(max_list) == 1:
                return max_list[0]
            
            else:
                return max_list[random.randint(0, len(max_list)-1)]

# Conclusion

Implementation of the artificial intelligence algorithms + optimization heuristics were successful. Further developments of this algorithm can be used for the following applications:

- Portfolio optimization: Given certain constraints, select the best combination of financial assets to purchase